In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import warnings
warnings.filterwarnings('ignore')

In [68]:
 df = pd.read_excel('OnlineRetail.xlsx')

In [4]:
df.shape

(541909, 8)

In [6]:
df.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
31506,538991,22910,PAPER CHAIN KIT VINTAGE CHRISTMAS,40,2010-12-15 11:53:00,2.55,17511.0,United Kingdom
43679,540113,22510,GINGHAM BABUSHKA DOORSTOP,8,2011-01-05 09:11:00,1.25,13126.0,United Kingdom
234343,557502,85131A,BEADED PEARL HEART WHITE ON STICK,1,2011-06-20 15:32:00,0.83,NaN,United Kingdom
510861,579470,82552,WASHROOM METAL SIGN,4,2011-11-29 15:06:00,1.45,16549.0,United Kingdom
495772,578323,82484,WOOD BLACK BOARD ANT WHITE FINISH,2,2011-11-23 17:19:00,7.95,15743.0,United Kingdom


In [8]:
df.dtypes

,0
InvoiceNo,object
StockCode,object
Description,object
Quantity,int64
InvoiceDate,datetime64[ns]
UnitPrice,float64
CustomerID,float64
Country,object


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [10]:
df.isnull().sum()

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


In [69]:
df.dropna(subset = 'Description', axis = 0, inplace = True)

In [14]:
df.shape

(540455, 8)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 540455 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    540455 non-null  object        
 1   StockCode    540455 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     540455 non-null  int64         
 4   InvoiceDate  540455 non-null  datetime64[ns]
 5   UnitPrice    540455 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      540455 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 37.1+ MB


In [17]:
## Eliminate white spaces in descriptionwith regular expression

In [70]:
import re

In [71]:
df['Description'] = df['Description'].astype(str).apply(lambda x: re.sub(r'\s+', '', x).strip()) # \s = space, + = 1 or more space character

In [72]:
df[df['InvoiceNo'].astype(str).str.contains('C')].sample(10)
#converting invoiceno to string first with type casting and then checking if any item contains c or cna use str.startswith('c'), c says in this data is cancelled transaction


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
38939,C539601,22768,FAMILYPHOTOFRAMECORNICE,-2,2010-12-20 13:58:00,9.95,14119.0,United Kingdom
396482,C571066,21658,GLASSBEURREDISH,-2,2011-10-13 15:05:00,3.95,12709.0,Germany
385506,C570203,71477,COLOURGLASS.START-LIGHTHOLDER,-2,2011-10-07 16:49:00,3.29,15311.0,United Kingdom
385513,C570203,21929,JUMBOBAGPINKVINTAGEPAISLEY,-1,2011-10-07 16:49:00,1.79,15311.0,United Kingdom
531353,C580919,23493,VINTAGEDOILYTRAVELSEWINGKIT,-4,2011-12-06 14:07:00,1.95,16571.0,United Kingdom
256308,C559486,22423,REGENCYCAKESTAND3TIER,-1,2011-07-08 13:51:00,12.75,13047.0,United Kingdom
6781,C536978,22776,SWEETHEARTCAKESTAND3TIER,-1,2010-12-03 14:19:00,9.95,17954.0,United Kingdom
19351,C537853,22697,GREENREGENCYTEACUPANDSAUCER,-1,2010-12-08 15:57:00,2.95,13370.0,United Kingdom
49859,C540535,20681,PINKPOLKADOTCHILDRENSUMBRELLA,-1,2011-01-09 14:17:00,3.25,15005.0,United Kingdom
352741,C567681,20726,LUNCHBAGWOODLAND,-2,2011-09-21 16:24:00,1.65,15433.0,United Kingdom


In [22]:
# leave the ids starting with c

In [73]:
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df= df[~ df['InvoiceNo'].str.contains('C')]
print(df.shape)

(531167, 8)


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 531167 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    531167 non-null  object        
 1   StockCode    531167 non-null  object        
 2   Description  531167 non-null  object        
 3   Quantity     531167 non-null  int64         
 4   InvoiceDate  531167 non-null  datetime64[ns]
 5   UnitPrice    531167 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      531167 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 36.5+ MB


In [75]:
df.Country.nunique()

38

In [76]:
print('No of unique countries:',df.Country.nunique())
df.Country.value_counts(normalize = True).head(6)

No of unique countries: 38


,proportion
Country,
United Kingdom,0.915283
Germany,0.017023
France,0.015829
EIRE,0.014862
Spain,0.004678
Netherlands,0.004449


We can see, majority of the proportion is from UK

In [36]:
# Let's see Germany: FOr market basket analysis, it makes sense to analyze region wise, the specific the location the accurate the measure

In [77]:
grouped_df = (df[df['Country']== 'Germany']# specifying the region
              .groupby(['InvoiceNo', 'Description'])
              .size()# count
              .unstack()#pivot method, coverts description into col.
              .fillna(0)
              .reset_index()#to tackle the filtering and cleaning
              .set_index('InvoiceNo'))# setting index on invoiceNo
print(grouped_df.shape)
grouped_df.head()

(457, 1692)


Description,10COLOURSPACEBOYPEN,12COLOUREDPARTYBALLOONS,12IVORYROSEPEGPLACESETTINGS,12MESSAGECARDSWITHENVELOPES,12PENCILSMALLTUBEWOODLAND,12PENCILSSMALLTUBEREDRETROSPOT,12PENCILSSMALLTUBESKULL,12PENCILSTALLTUBEPOSY,12PENCILSTALLTUBEREDRETROSPOT,12PENCILSTALLTUBESKULLS,12PENCILSTALLTUBEWOODLAND,12PINKHEN+CHICKSINBASKET,12REDROSEPEGPLACESETTINGS,16PIECECUTLERYSETPANTRYDESIGN,200BENDYSKULLSTRAWS,200RED+WHITEBENDYSTRAWS,20DOLLYPEGSRETROSPOT,2PICTUREBOOKEGGSEASTERBUNNY,2PICTUREBOOKEGGSEASTERCHICKS,36DOILIESDOLLYGIRL,36FOILHEARTCAKECASES,36FOILSTARCAKECASES,36PENCILSTUBEREDRETROSPOT,36PENCILSTUBESKULLS,36PENCILSTUBEWOODLAND,3DCHRISTMASSTAMPSSTICKERS,3DRAWERANTIQUEWHITEWOODCABINET,3DTRADITIONALCHRISTMASSTICKERS,3DVINTAGECHRISTMASSTICKERS,3HOOKHANGERMAGICGARDEN,3HOOKPHOTOSHELFANTIQUEWHITE,3PIECESPACEBOYCOOKIECUTTERSET,3RAFFIARIBBONS50'SCHRISTMAS,3STRIPEYMICEFELTCRAFT,3TIERCAKETINGREENANDCREAM,3TIERCAKETINREDANDCREAM,3TIERSWEETHEARTGARDENSHELF,3TRADITIONAlBISCUITCUTTERSSET,4LAVENDERBOTANICALDINNERCANDLES,4TRADITIONALSPINNINGTOPS,50'SCHRISTMASGIFTBAGLARGE,50'SCHRISTMASPAPERGIFTBAG,5HOOKHANGERMAGICTOADSTOOL,5HOOKHANGERREDMAGICTOADSTOOL,60CAKECASESDOLLYGIRLDESIGN,60CAKECASESVINTAGECHRISTMAS,60TEATIMEFAIRYCAKECASES,6GIFTTAGS50'SCHRISTMAS,6GIFTTAGSVINTAGECHRISTMAS,6RIBBONSELEGANTCHRISTMAS,...,WOODS/3CABINETANTWHITEFINISH,WOODSTAMPSETBESTWISHES,WOODSTAMPSETFLOWERS,WOODSTAMPSETHAPPYBIRTHDAY,WOODSTAMPSETTHANKYOU,WORLDWAR2GLIDERSASSTDDESIGNS,"WRAP,BILLBOARDFONTSDESIGN",WRAP50'SCHRISTMAS,WRAPALPHABETDESIGN,WRAPALPHABETPOSTER,WRAPBADHAIRDAY,WRAPBILLBOARDFONTSDESIGN,WRAPBIRDGARDEN,WRAPCAROUSEL,WRAPCHRISTMASSCREENPRINT,WRAPCIRCUSPARADE,WRAPCOWBOYS,WRAPDOILEYDESIGN,WRAPDOLLYGIRL,WRAPENGLISHROSE,WRAPFLOWERSHOP,WRAPFOLKART,WRAPGINGHAMROSE,WRAPGREENPEARS,WRAPILOVELONDON,WRAPMAGICFOREST,WRAPMONSTERFUN,WRAPPAISLEYPARK,WRAPPINKFAIRYCAKES,WRAPPOPPIESDESIGN,WRAPREDAPPLES,WRAPREDDOILEY,WRAPREDVINTAGEDOILY,WRAPSUKIANDFRIENDS,WRAPVINTAGEDOILY,WRAPVINTAGELEAFDESIGN,WRAPVINTAGEPETALSDESIGN,WRAPWEDDINGDAY,YELLOWCOATRACKPARISFASHION,YOU'RECONFUSINGMEMETALSIGN,YULETIDEIMAGESGIFTWRAPSET,ZINCBOXSIGNHOME,ZINCFOLKARTSLEIGHBELLS,ZINCHEARTLATTICET-LIGHTHOLDER,ZINCHEARTT-LIGHTHOLDER,ZINCMETALHEARTDECORATION,ZINCSTART-LIGHTHOLDER,ZINCT-LIGHTHOLDERSTARLARGE,ZINCT-LIGHTHOLDERSTARSSMALL,ZINCWILLIEWINKIECANDLESTICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [38]:
# above is shoing the description vs invoice, and where ever there is any item purchased, it is shows 1 value

In [78]:
def custom_encoding(x):
  return 1 if x>=1 else 0

grouped_df = grouped_df.applymap(custom_encoding)

In [79]:
frequentitems = apriori(grouped_df, min_support = 0.05, use_colnames = True)

In [80]:
num_itemsets = len(frequentitems)

In [81]:
rules = association_rules(frequentitems, metric = 'confidence', min_threshold = 0.6, num_itemsets = num_itemsets)

In [82]:
rules.sort_values(  by = 'lift', ascending = False).sample(6)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
38,"(PLASTERSINTINWOODLANDANIMALS, PLASTERSINTINCI...",(POSTAGE),0.067834,0.818381,0.054705,0.806452,0.985423,1.0,-0.000809,0.938366,-0.015621,0.065789,-0.065682,0.436648
9,(JUMBOBAGREDRETROSPOT),(POSTAGE),0.078775,0.818381,0.063457,0.805556,0.984329,1.0,-0.001010,0.934042,-0.016989,0.076115,-0.070616,0.441548
7,(JAMMAKINGSETPRINTED),(POSTAGE),0.091904,0.818381,0.074398,0.809524,0.989177,1.0,-0.000814,0.953501,-0.011905,0.089005,-0.048766,0.450216
25,(RETROSPOTPARTYBAG+STICKERSET),(POSTAGE),0.061269,0.818381,0.050328,0.821429,1.003724,1.0,0.000187,1.017068,0.003953,0.060686,0.016781,0.441463
45,"(ROUNDSNACKBOXESSETOF4FRUITS, ROUNDSNACKBOXESS...",(POSTAGE),0.131291,0.818381,0.124726,0.950000,1.160829,1.0,0.017280,3.632385,0.159486,0.151194,0.724699,0.551203
48,"(ROUNDSNACKBOXESSETOF4WOODLAND, SPACEBOYLUNCHBOX)",(POSTAGE),0.070022,0.818381,0.061269,0.875000,1.069184,1.0,0.003965,1.452954,0.069580,0.074074,0.311747,0.474933


The postage is kinda like if you have done shopping and while billing bought a carry bag by paying money, that is postage, so people do not go to buy postage. so delete postages. Have to re-run the entire code from the aprori

In [83]:
def custom_encoding(x):
  return 1 if x>=1 else 0

grouped_df = grouped_df.applymap(custom_encoding)
grouped_df.drop('POSTAGE', inplace = True, axis = 1)

In [84]:
frequentitems = apriori(grouped_df, min_support = 0.05, use_colnames = True)

In [86]:
rules = association_rules(frequentitems, metric = 'confidence', min_threshold = 0.6, num_itemsets = num_itemsets)

In [87]:
rules.sort_values(  by = 'lift', ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(REDRETROSPOTCHARLOTTEBAG),(WOODLANDCHARLOTTEBAG),0.070022,0.126915,0.059081,0.843750,6.648168,1.0,0.050194,5.587746,0.913551,0.428571,0.821037,0.654634
1,(ROUNDSNACKBOXESSETOF4FRUITS),(ROUNDSNACKBOXESSETOF4WOODLAND),0.157549,0.245077,0.131291,0.833333,3.400298,1.0,0.092679,4.529540,0.837922,0.483871,0.779227,0.684524
2,(SPACEBOYLUNCHBOX),(ROUNDSNACKBOXESSETOF4WOODLAND),0.102845,0.245077,0.070022,0.680851,2.778116,1.0,0.044817,2.365427,0.713415,0.251969,0.577243,0.483283
